In [1]:
import cv2
import numpy as np
import math
import colorsys
import csv

In [2]:
#ランダムで座標と値を発生
#本番はリストで受け取るので、この関数は不要
def getRandom2DPoints(width, height, n):
    with open('sst.csv','r') as f:
        reader = csv.reader(f)
        pts = np.zeros((50,3))
        i = 0
        for line in reader:
            if(i>0):
                pts[i-1][0]=line[0]
                pts[i-1][1]=line[1]
                pts[i-1][2]=line[2]
            i=i+1
        return pts

In [3]:
def get_color2(a1,a2,a3):
    #平均値を求める
    ave = (a1+a2+a3)/3
    if (ave>=0):
        r = 255
        g = 255 - ave*5
        b = 255 - ave*5
    if (ave<0):
        r = 255 + ave*5
        g = 255 + ave*5
        b = 255
    return(b,g,r)

In [4]:
def DelaunayDiagram(imgd, subdiv,array):
    height, width = imgd.shape[:2]

    # ドロネーの三角形
    triangles = subdiv.getTriangleList()
    pols = triangles.reshape(-1, 3, 2)
    inner_pts = np.empty((0, 3, 2))
    
    for pa in pols:
        # フレーム外の点は除外
        cnt = 0
        for p in pa:
            if p[0] < 0 or width <= p[0] :
                break
            if p[1] < 0 or height <= p[1] :
                break
            cnt += 1
        # フレーム内の点は有効
        if cnt == 3:
            inner_pts = np.append(inner_pts, [pa], axis=0)
    
    #rgb値を格納する配列を作成
    color_list = np.zeros((len(inner_pts),3))
    
    #3値を渡して色を設定
    for i in range(len(inner_pts)):
        color_list[i]=get_color2(array[(int)(inner_pts[i][0][0])][(int)(inner_pts[i][0][1])],array[(int)(inner_pts[i][1][0])][(int)(inner_pts[i][1][1])],array[(int)(inner_pts[i][2][0])][(int)(inner_pts[i][2][1])])
    #図形の描画
    for i in range(len(inner_pts)):
        #三角形の描画
        cv2.fillPoly(imgd, [inner_pts[i].astype(int)], (color_list[i][0],color_list[i][1],color_list[i][2]), 1)
        #三角形の辺の描画
        cv2.line(imgd,(inner_pts[i][0][0].astype(int),inner_pts[i][0][1].astype(int)),(inner_pts[i][1][0].astype(int),inner_pts[i][1][1].astype(int)),(0,0,0))
        cv2.line(imgd,(inner_pts[i][1][0].astype(int),inner_pts[i][1][1].astype(int)),(inner_pts[i][2][0].astype(int),inner_pts[i][2][1].astype(int)),(0,0,0))
        cv2.line(imgd,(inner_pts[i][2][0].astype(int),inner_pts[i][2][1].astype(int)),(inner_pts[i][0][0].astype(int),inner_pts[i][0][1].astype(int)),(0,0,0))

In [5]:
def main():

    width = 300
    height = 300
    
    #点の生成
    pts = getRandom2DPoints(width,height,50)
    print(pts)
    #白紙を作る
    img = np.full((height, width, 3), 255, np.uint8)
    
    rect = (0, 0, width, height)
    subdiv = cv2.Subdiv2D(rect)

    #(x1,y1,a1)の時、array[x1][y1]=a1とする
    array = np.zeros((width,height))
    #print(array)
    
    for p in pts:
        subdiv.insert((int(p[0]), int(p[1])))
        array[int(p[0])][int(p[1])] = p[2]
        
    DelaunayDiagram(img, subdiv,array)

    for p in pts:
        #color = p[2]
        cv2.circle(img, (int(p[0]),int(p[1])), 4, (0, 0, 0), thickness=-1, lineType=cv2.LINE_AA)
   
    cv2.imwrite('test3.png', img)

In [6]:
if __name__ == '__main__':
    main()

[[ 45.         188.49998474  10.64608123]
 [156.875       94.50000763  27.17641421]
 [250.          65.          28.42566308]
 [275.625      169.50000763  22.53485352]
 [269.375      242.5          9.14593718]
 [251.25       201.50001526  13.33275037]
 [278.75       140.          25.86444153]
 [276.25       222.99999237  11.0511724 ]
 [141.25        56.00000381  28.61383446]
 [ 82.5        247.00000763  -4.1671879 ]
 [ 19.375       89.50000763  22.78574869]
 [196.875      174.00001526  19.03538856]
 [ 38.125      147.99999237  19.75409869]
 [180.          76.49999619  28.14601951]
 [278.125      172.99999237  22.39633847]
 [250.         149.00001526  26.31657554]
 [165.          21.49999619  28.89609153]
 [108.125      216.50001526  -1.1668998 ]
 [112.5         46.00000381  27.92648623]
 [138.125       71.49999619  27.7226339 ]
 [ 80.625      167.00000763   8.14496998]
 [176.875       46.00000381  28.78632489]
 [270.625      150.          24.45838317]
 [ 12.5        105.          22.30